In [3]:
pip install numpy pandas matplotlib plotly seaborn jupyter_contrib_nbextensions

Note: you may need to restart the kernel to use updated packages.


In [8]:
import csv
import numpy as np
import pandas as pd
import matplotlib
import plotly
import seaborn as sns
import tensorflow as tf
import tensorflow.keras
import jupyter_contrib_nbextensions

# 1. DATA LOADING & INSPECTION

In [5]:
df = pd.read_csv("oneskydata.csv")

In [7]:
df.info()
# id - reading;
# device_id - various devices
# pm25 - particles less than 2.5
# db - decibels
# co2 - carbon dioxide 
# voc - gases (volatile organic compounds)
# no2 - nitrogen
# co - carbon monoxide
# pm1 - 1 micrometer
# pm10 - 10 micrometers
# humidity - percent
# battery - device battery (possibly in mAh)
# sv - possibly software version ( ex: 0.01.18 --> 0.01.19)
# ps - possibly power status (cherged/not-charged)
# latitude & longitude - location, however missing from some readings
# message time - possibly time when information is received, whereas
# reading time - is possible when information is read

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58767 entries, 0 to 58766
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            58767 non-null  int64  
 1   device_id     58767 non-null  object 
 2   reading_time  58767 non-null  object 
 3   pm25          57835 non-null  float64
 4   db            57835 non-null  float64
 5   co2           57835 non-null  float64
 6   voc           57835 non-null  float64
 7   no2           57835 non-null  float64
 8   co            57835 non-null  float64
 9   no            57835 non-null  float64
 10  pm1           57835 non-null  float64
 11  pm10          57835 non-null  float64
 12  humidity      57835 non-null  float64
 13  temprature    57835 non-null  float64
 14  pressure      57835 non-null  float64
 15  battery       57835 non-null  float64
 16  signal        58767 non-null  int64  
 17  sv            58767 non-null  object 
 18  ps            58767 non-nu

In [14]:
df.agg(
    {
        "pm25": ["min", "max", "median", "skew"],
        
    }
)

,pm25
min,0.000000
max,802.000000
median,6.000000
skew,4.227023


# 2. DATA PREPROCESSING

In [17]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57835 entries, 0 to 58766
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            57835 non-null  int64  
 1   device_id     57835 non-null  object 
 2   reading_time  57835 non-null  object 
 3   pm25          57835 non-null  float64
 4   db            57835 non-null  float64
 5   co2           57835 non-null  float64
 6   voc           57835 non-null  float64
 7   no2           57835 non-null  float64
 8   co            57835 non-null  float64
 9   no            57835 non-null  float64
 10  pm1           57835 non-null  float64
 11  pm10          57835 non-null  float64
 12  humidity      57835 non-null  float64
 13  temprature    57835 non-null  float64
 14  pressure      57835 non-null  float64
 15  battery       57835 non-null  float64
 16  signal        57835 non-null  int64  
 17  sv            57835 non-null  object 
 18  ps            57835 non-null  b

In [18]:
#deleting cells with comma (no location)
mask = df.apply(lambda x: x.str.contains(',') if x.dtype == "object" else False, axis=1).any(axis=1)
df = df[~mask]

print(df)

             id     device_id             reading_time   pm25    db  co2   
100    10501451  4C11AEADF0DC  2023-10-09 08:00:01.000  279.0   0.0  0.0  \
101    10501452  4C11AEADF0DC  2023-10-09 08:01:01.000  283.0   0.0  0.0   
102    10501453  4C11AEADF0DC  2023-10-09 08:02:01.000  292.0   0.0  0.0   
104    10501461  4C11AEADF0DC  2023-10-09 08:05:01.000  293.0   0.0  0.0   
105    10501462  4C11AEADF0DC  2023-10-09 08:06:01.000  299.0   0.0  0.0   
...         ...           ...                      ...    ...   ...  ...   
58762   6860576  E831CD848A1C  2023-01-24 08:16:01.000   22.0  59.0  0.0   
58763   6860577  E831CD848A1C  2023-01-24 08:17:01.000   23.0  58.0  0.0   
58764   6860578  E831CD848A1C  2023-01-24 08:18:01.000   22.0  56.0  0.0   
58765   6860645  E831CD848A1C  2023-01-24 08:25:01.000   23.0  57.0  0.0   
58766   6860648  E831CD848A1C  2023-01-24 08:28:01.000   19.0  55.0  0.0   

         voc  no2   co   no  ...  humidity  temprature  pressure  battery   
100    125

In [29]:
# convert dates to datetime objects

#df['reading_time'] = pd.to_datetime(df['reading_time'])
df.loc[:, 'reading_time'] = pd.to_datetime(df['reading_time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43330 entries, 100 to 58766
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            43330 non-null  int64         
 1   device_id     43330 non-null  object        
 2   reading_time  43330 non-null  datetime64[ns]
 3   pm25          43330 non-null  float64       
 4   db            43330 non-null  float64       
 5   co2           43330 non-null  float64       
 6   voc           43330 non-null  float64       
 7   no2           43330 non-null  float64       
 8   co            43330 non-null  float64       
 9   no            43330 non-null  float64       
 10  pm1           43330 non-null  float64       
 11  pm10          43330 non-null  float64       
 12  humidity      43330 non-null  float64       
 13  temprature    43330 non-null  float64       
 14  pressure      43330 non-null  float64       
 15  battery       43330 non-null  float64  

In [33]:
# uncheck if warning about slices is given/not converting to datetime64
# df['message_time'] = pd.to_datetime(df['message_time']) 
df.loc[:, 'message_time'] = pd.to_datetime(df['message_time'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43330 entries, 100 to 58766
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            43330 non-null  int64         
 1   device_id     43330 non-null  object        
 2   reading_time  43330 non-null  datetime64[ns]
 3   pm25          43330 non-null  float64       
 4   db            43330 non-null  float64       
 5   co2           43330 non-null  float64       
 6   voc           43330 non-null  float64       
 7   no2           43330 non-null  float64       
 8   co            43330 non-null  float64       
 9   no            43330 non-null  float64       
 10  pm1           43330 non-null  float64       
 11  pm10          43330 non-null  float64       
 12  humidity      43330 non-null  float64       
 13  temprature    43330 non-null  float64       
 14  pressure      43330 non-null  float64       
 15  battery       43330 non-null  float64  

# 3. EXPLORING THE DATASET 

TODO: 
    1. summaries for pollutants/histograms/anything that makes a significant point on their distribution
    2. time series analysis?? 

# Task2 – Develop a list of data sets required within the new customer portal

Dataset 1: Air quality readings
- ID
- PM1
- PM2.5
- PM10
- CO
- CO2
- NO
- NO2
- VOC

Dataset 2: Environment readings
- ID
- Humidity
- Temperature
- Pressure 

Dataset 3: Device Data
- ID
- Device ID
- Mode(ps) - maybe power? 
- Software version (assuming sv is it) 

Dataset 4: Geolocation Data
- ID
- Longitute
- Latitude
- Decibels (sound pollution related to geo)

Dataset 5: Event Logging 
- ID 
- Reading Time
- Message Time


# 4. DATA VISUALISATION

TODO: 1. seaborn, matplotlib (scatter plots (on different levels of particles), bar charts...) on Ch3 points 2. geopy for location 3. time series? again for showing the changes over time


# TASK 3 - Develop a list of visuals required within the new customer portal

1. Time Series Analysis - air quality metrics over time in that location (if above WHO standard)
- ID
- PM1 - n/a 
- PM2.5 - 5 (24h mean)
- PM10 - 15 (24h mean)
- CO - 35 (1h mean)
- CO2 - n/a
- NO & No2- 200 (1h mean)
- VOC - N/a
- longitute
- latitude

Update 01.13/16h
The Pm1, Pm2.5 & Pm10 is in union, meaning pm1 is a subset of Pm2.5 and subset of pm10. Thus, I will drop pm1 and pm10, focusing on the middle subset, which has the highest chance to represent the category as a whole. 
I will drop Co2, as no guideline where it becomes bad by WHO has been found
I will add no and no2 together as they are usually referred together and will create a new column named nitrogen
I will drop VOC as even though it's harmful, I could not find the exact guideline based on regulations
I will also drop non location entries, empty entries, anything that does not help establishing a prediction over the location

Thus the updated parameters:
- ID
- reading_time
- pm25
- co
- nitrogen 
- longitute
- latitude


2. Heat Map where the pollution is biggest based on location
3. Overlaying heat maps with air quality metrics?? 
4. Data dashboard - summary of key metrics

# 5. ANY FANCY ANALYSIS 

TODO POSSIBLY: 1. correlation between particles & heatmap on findings 2. Other datasets? such as investigating how environment is correlated to these particle trends